In [51]:
import pandas as pd
import numpy as np
import matplotlib as mpl # optional (here)
import matplotlib.pyplot as plt
# import seaborn as sns # Optional, will only affect the color of bars and the grid

from ipywidgets import widgets, interactive

from io import StringIO
import io
import cv2

from ProcessImage import ProcessImage
pimg = ProcessImage()
pimg.main()
pimg.load((7,))
size = len(pimg.segList.BUSList)
seg = pimg.segList.BUSList[size -1]
seg.findContours(addImages=True)

imagem = cv2.bitwise_not(seg.image)
is_success, im_buf_arr = cv2.imencode(".png", imagem)
byte_im = im_buf_arr.tobytes()

buf = io.BytesIO()
seg.PILimage.save(buf, format='png')
byte_orig = buf.getvalue()


imageOrig = widgets.Image(
    value=byte_orig,
    format='png',
    width=400,
    height=300,
)

image1 = widgets.Image(
    value=byte_im,
    format='png',
    width=400,
    height=300,
)

opts = list(seg.images.keys())

imageSelect = widgets.Dropdown(
    options=opts,
    value=opts[0],
    description='Images:',
    disabled=False,
)

def setup_ui(df):

    out = widgets.Output()
    with out:
        display(df)
    return out

out = setup_ui(seg.contourStats)

output2 = widgets.Output()

def on_value_change(change):
    with output2:
        print(change['new'])
    with image1:
        key = change['new']
        is_success, im_buf_arr = cv2.imencode(".png", seg.images.get(key).get('image'))
        byte_im = im_buf_arr.tobytes()
        value = byte_im
        diplay(image1)

imageSelect.observe(on_value_change, names='value')

display(imageSelect, image1, output2)
# settings to display all columns
# pd.set_option("display.max_columns", None)
# display the dataframe head


testdata=StringIO("""Year,Sex,Area,Count
2015,W,Dhaka,6
2015,M,Dhaka,3
2015,W,Khulna,1
2015,M,Khulna,8
2014,M,Dhaka,13
2014,W,Dhaka,20
2014,M,Khulna,9
2014,W,Khulna,6
2013,W,Dhaka,11
2013,M,Dhaka,2
2013,W,Khulna,8
2013,M,Khulna,5
2012,M,Dhaka,12
2012,W,Dhaka,4
2012,W,Khulna,7
2012,M,Khulna,1
    """)

df = pd.read_csv(testdata, sep=",")

# Create two bounded text box that allow only numbers between the min year (2012) and the max year (2015)
start_year = widgets.BoundedFloatText(
    value=df.Year.min(),
    min=df.Year.min(),
    max=df.Year.max(),
    step=1,
    description='Start Year:',
    disabled=False,
    color='black'
)
end_year = widgets.BoundedFloatText(
    value=df.Year.max(),
    min=df.Year.min(),
    max=df.Year.max(),
    step=1,
    description='End Year:',
    disabled=False,
    color='black'
)

# Make a dropdown to select the Area, or "All"
area = widgets.Dropdown(
    options=['All'] + list(df['Area'].unique()),
    value='All',
    description='Area:',
)

def plotit(area, start_year, end_year):
    """
    Filters and plot the dataframe as a stacked bar chart of count of Male versus Women

    Args:
    -----
        * area (str): the area to filter on, or "All" to display all Areas

        * start_year, end_year (int, as float): the start and ends years, inclusive

        Note: the dataframe to plot is globally defined here as `df`

    Returns:
    --------
        A matplotlib stacked bar chart

    """
    if start_year > end_year:
        print("You must select a start year that is prior to end year")
    else:
        df2 = df.copy()
        if area != 'All':
            df2 = df2[df2.Area == area]

        # Filter between min and max years (inclusive)
        df2 = df2[(df2.Year >= start_year) & (df2.Year <= end_year)]


        # Plot it (only if there's data to plot)
        if len(df2) > 0:
            df2.groupby(['Year', 'Sex']).sum()['Count'].unstack().plot(kind='bar', stacked=True, title="Area = {}".format(area))
            plt.show();
        else:
            print("No data to show for current selection")

# interactive(plotit,start_year=start_year, area=area,  end_year=end_year)

segList size=38
Total number of contours detected: 1223
Numbers of contours plotted=7


C:\Users\djhalama\Documents\GitHub\BUS\Segmentation\BUSSegmentor.py:227: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.sort_values(by=['mean_val'], ascending=False, inplace=True)


Dropdown(description='Images:', options=('Original', 'GT', 'Gray Inverted with GT', 'Binary', 'With Contours',…

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02\x10\x00\x00\x01\x9c\x08\x00\x00\x00\x00\xad\xf1\…

Output()

In [30]:
imageOrig.keys
# imageOrig.layout

['_dom_classes',
 '_model_module',
 '_model_module_version',
 '_model_name',
 '_view_count',
 '_view_module',
 '_view_module_version',
 '_view_name',
 'format',
 'height',
 'layout',
 'value',
 'width']

In [37]:
opts

['Original',
 'GT',
 'Gray Inverted with GT',
 'Binary',
 'With Contours',
 'WithFilteredContours']

In [46]:
seg.contourStats


,id,imageName,cnt_id,aspect_ratio,area,areaFracGT,leftx,rightx,topy,bottomy,width,height,extent,solidity,angle,min_val,max_val,min_loc,max_loc,mean_val,cx,cy,perimeter,cnt,dist
0,7,000007.png,0,8.0,0.0,0.0,520,528,411,412,8,1,0.0,0.0,0.0,235.0,239.0,"(520, 411)","(527, 411)",237.125000,NaN,NaN,14.0,"[[[520, 411]], [[521, 411]], [[522, 411]], [[5...",NaN
1,7,000007.png,1,11.0,0.0,0.0,508,519,411,412,11,1,0.0,0.0,0.0,232.0,237.0,"(518, 411)","(511, 411)",234.909091,NaN,NaN,20.0,"[[[508, 411]], [[509, 411]], [[510, 411]], [[5...",NaN
2,7,000007.png,2,4.0,0.0,0.0,503,507,411,412,4,1,0.0,0.0,0.0,232.0,239.0,"(503, 411)","(505, 411)",235.000000,NaN,NaN,6.0,"[[[503, 411]], [[504, 411]], [[505, 411]], [[5...",NaN
3,7,000007.png,3,18.0,0.0,0.0,484,502,411,412,18,1,0.0,0.0,0.0,232.0,240.0,"(490, 411)","(495, 411)",236.111111,NaN,NaN,34.0,"[[[484, 411]], [[485, 411]], [[486, 411]], [[4...",NaN
4,7,000007.png,4,6.0,0.0,0.0,477,483,411,412,6,1,0.0,0.0,0.0,232.0,237.0,"(482, 411)","(480, 411)",234.833333,NaN,NaN,10.0,"[[[477, 411]], [[478, 411]], [[479, 411]], [[4...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1218,7,000007.png,1218,29.0,0.0,0.0,67,96,0,1,29,1,0.0,0.0,0.0,235.0,250.0,"(83, 0)","(90, 0)",239.793103,NaN,NaN,56.0,"[[[67, 0]], [[68, 0]], [[69, 0]], [[70, 0]], [...",NaN
1219,7,000007.png,1219,3.0,0.0,0.0,43,46,0,1,3,1,0.0,0.0,NaN,235.0,240.0,"(43, 0)","(44, 0)",237.000000,NaN,NaN,4.0,"[[[43, 0]], [[44, 0]], [[45, 0]], [[44, 0]]]",NaN
1220,7,000007.png,1220,23.0,0.0,0.0,17,40,0,1,23,1,0.0,0.0,0.0,233.0,249.0,"(39, 0)","(30, 0)",241.043478,NaN,NaN,44.0,"[[[17, 0]], [[18, 0]], [[19, 0]], [[20, 0]], [...",NaN
1221,7,000007.png,1221,5.0,0.0,0.0,11,16,0,1,5,1,0.0,0.0,0.0,234.0,248.0,"(15, 0)","(13, 0)",240.400000,NaN,NaN,8.0,"[[[11, 0]], [[12, 0]], [[13, 0]], [[14, 0]], [...",NaN
